In [4]:
import numpy as np
import cv2 

In [6]:
def region_interes(img, vertices):
    mask = np.zeros_like(img)
    match_mask_color = 255
    cv2.fillPoly(mask, vertices, match_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    cv2.imshow('Masked', masked_image)
    return masked_image

def linea_guia(img, lines):
    img = np.copy(img)
    blank_image = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(blank_image, (x1,y1), (x2,y2), (0, 255, 0), thickness=10)

    img = cv2.addWeighted(img, 0.8, blank_image, 1, 0.0)
    return img

def frame(img):
    new_width = int(img.shape[1]/1.5)
    new_height = int(img.shape[0]/1.5)
    
    region_of_interest_vertices = [
    (0, new_height),
    (new_width/2 - new_width/8, new_height/2),
    (new_width - new_width / 2,  new_height/2),
    (new_width, new_height)
    ]
    img = cv2.resize(img, (new_width, new_height))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray,50,150,apertureSize = 3)
    cropped_image = region_interes(edges,
                np.array([region_of_interest_vertices], np.int32),)
    lines = cv2.HoughLinesP(cropped_image,
                            rho=1,
                            theta=np.pi/180,
                            threshold=40,
                            lines=np.array([]),
                            minLineLength=30,
                            maxLineGap=5)
    image_with_lines = linea_guia(img, lines)
    return image_with_lines

cap = cv2.VideoCapture('test_video.mp4')
while cap.isOpened():
    ret, img= cap.read()

    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    img_lines = frame(img)

    cv2.imshow('frame', img_lines)
    if cv2.waitKey(1) == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()

Can't receive frame (stream end?). Exiting ...
